# Juan Carlos Perez Ramirez
## Procesamiento de Lenguaje Natural
## Tarea 5: Modelos de Lenguaje Neuronales

In [1]:
# General
import os
import time
import shutil
import random
from typing import Tuple
from argparse import Namespace  # Guardar variables y parámetros
import matplotlib.pyplot as plt

# Preprocesamiento
import nltk
from nltk.corpus import stopwords
from nltk import ngrams
from nltk.tokenize import TweetTokenizer
from nltk import FreqDist
import numpy as np
import pandas as pd

# PyTorch
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.nn.functional as F

# scikit-learn
from sklearn.metrics import accuracy_score

# tqmd
from tqdm.notebook import tqdm

## Carga de los embeddings preentrenados

In [2]:
embs_df = pd.read_csv("word2vec_col.txt", skiprows=1, header=None, sep=' ')
embs_df.set_index(0,inplace=True)
embs_df.head()

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,,,,,,,,,,,,,,,,,,,,,
de,-1.641680,1.447671,-2.283216,-1.965226,-0.222943,5.105217,-0.120701,-0.126822,-3.177338,-3.454396,...,0.399476,0.887398,-1.994891,2.912205,3.257981,-2.599953,-1.995134,-0.176465,3.946902,-2.792494
que,2.167917,-2.581055,-3.290037,-2.788440,1.930902,-2.482932,-2.540504,0.918257,-0.375576,-4.981284,...,2.166590,0.142203,1.698710,-1.105366,6.916267,-3.977985,3.989532,-1.923247,2.035671,2.539047
la,0.727598,-3.604198,0.793098,-4.076829,-0.050293,8.435444,-2.233952,2.568616,-2.221771,-1.738045,...,-2.224793,6.436301,-5.083917,-3.572159,3.077797,1.510428,0.011997,2.240481,0.825278,-5.749021
a,4.178944,1.255945,-0.714395,0.245530,-4.155548,1.866886,0.096566,1.327916,2.054787,0.943383,...,3.369879,3.655371,-1.750744,2.508173,6.827713,-5.676323,1.920513,-1.108223,3.379427,-1.197493
y,0.799879,-0.130996,-1.711219,-0.395188,-0.195817,2.218104,-0.453679,2.695769,-4.302141,-3.187639,...,1.321977,3.195324,-3.616187,0.961742,4.989526,-4.719704,0.692478,-1.918830,2.808852,0.732657


Se convierte el dataframe a un diccionario

In [3]:
embs_dict = embs_df.apply(lambda row: row.tolist(), axis=1).to_dict()

In [4]:
seed = 1111
random.seed(seed)  # python seed
np.random.seed(seed)  # numpy seed
torch.manual_seed(seed)  # torch seed
torch.backends.cudnn.benchmark = False

In [5]:
def get_texts_from_file(path_corpus, path_truth):
  """
  Función para leer los archivos de tuits. Cada línea (tuit) será un elemento de la lista.
  """
  tr_txt = []
  tr_y = []
  with open(path_corpus) as f_corpus, open(path_truth) as f_truth:
    for tweet in f_corpus:
      tr_txt.append(tweet)
    for label in f_truth:
      tr_y.append(label)
  return tr_txt, tr_y


PATH = "../../corpus/"

X_train, y_train = get_texts_from_file(
  PATH + "/mex20_train.txt",
  PATH + "/mex20_train_labels.txt")

X_val, y_val = get_texts_from_file(
  PATH + "/mex20_val.txt",
  PATH + "/mex20_val_labels.txt")

y_train = list(map(int, y_train))
y_val = list(map(int, y_val))

In [6]:
args = Namespace()
args.N = 4

In [53]:
class NgramData():
  """
  Toma un corpus y a través de los métodos fit y transform, se crea una lista de 
  n-gramas pensada para el entrenamiento de la CBOW.
  """

  def __init__(self,
               N: int,
               vocab_max: int = 5000,
               tokenizer: callable = None,
               embeddings: dict = None):
    """
    Constructor de la clase.

    Args:
        N (int): Tamaño de los n-gramas.
        vocab_max (int, optional): Tamaño máximo del vocabulario a considerar. Defaults to 5000.
        tokenizier (callable, optional): Tokenizador. Defaults to None.
        embeddings (np.ndarray, optional): Matriz de embeddings pre-entrenada. Debe entrar en el orden en el que entran las palabras. Defaults to None.
    """
    self.N = N
    self.vocab_max = vocab_max
    self.tokenizer = tokenizer if tokenizer else self.default_tokenizer
    self.embeddings = embeddings

    if self.embeddings:
      self.embedding_size = len(list(embeddings.values())[0])

    # tokens a ignorar
    self.punct = ['.', ',', ';', ':', '-', '^', '"',
                  '"', '!', '¡', '¿', '?', '<url>', '#', '@usuario',
                  '\n', '\t', '(', ')', "'", '"', '$', '&', '*', '@', '_', '<', '>', '/', '“', '”']

    # tokens especiales
    self.UNK = "<unk>"
    self.SOS = "<s>"
    self.EOS = "</s>"

  def get_vocab_size(self) -> int:
    """
    Devuelve el tamaño del vocabulario.

    Returns:
        int: Tamaño del vocabulario.
    """
    return len(self.vocab)

  def default_tokenizer(self, doc: str) -> list:
    """
    Tokenizador por defecto. Simplemente separa cada oración por espacios.

    Args:
        doc (str): Documento a tokenizar.

    Returns:
        list: Lista de tokens.
    """
    return doc.split(" ")

  def remove_word(self, word: str) -> bool:
    """
    Verifica si la palabra en cuestión debe eliminarse según los siguientes criterios:
    - Es un signo de puntuación
    - Es un dígito

    Args:
        word (str): Palabra a evaluar.

    Returns:
        bool: True si se elimina.
    """
    word = word.lower()
    is_punct = True if word in self.punct else False
    is_digit = word.isnumeric()
    return is_punct or is_digit

  def sortFreqDist(self, freq_dist: nltk.FreqDist) -> list:
    """
    Devuelve una lista con el top de palabras por frecuencia. El tamaño de la lista es self.vocab_max.

    Args:
        freq_dist (nltk.FreqDist): Objeto de frecuencias (nltk) del corpus considerado.

    Returns:
        list: Lista de tamaño self.vocab_max.
    """
    freq_dist = dict(freq_dist)
    # Aquí key es una función que se aplica a cada parámetro
    # antes de compararlo. En este caso se pasa
    # freq_dist.get para asegurarse de que el ordenamiento
    # se haga por las frecuencias y no por orden alfabético.
    return sorted(freq_dist,
                  key=freq_dist.get,
                  reverse=True)

  def get_vocab(self, corpus: list[str]) -> set:
    """
    Devuelve el vocabulario a partir de un corpus dado.

    Args:
        corpus (list[str]): Corpus del cual se quiere obtener el vocabulario. Lista de documentos.

    Returns:
        set: Vocabulario.
    """
    freq_dist = FreqDist(
      [w.lower()
       for sentence in corpus
       for w in self.tokenizer(sentence)
       if not self.remove_word(w)]
    )
    sorted_words = self.sortFreqDist(freq_dist)[:self.vocab_max-3]
    return set(sorted_words)

  def fit(self, corpus: list[str]) -> None:
    """
    Carga el vocabulario y crea diccionarios de índices <-> palabras. Además, si se aporta una matriz de embeddings pre-entrenados, también construye la submatriz con los elementos del vocabulario.

    Args:
        corpus (list[str]): Lista de documentos.
    """
    # Cargamos el vocabulario
    self.vocab = self.get_vocab(corpus)
    self.vocab.add(self.UNK)
    self.vocab.add(self.SOS)
    self.vocab.add(self.EOS)

    # Diccionarios palabras <-> ids
    self.w2id = dict()
    self.id2w = dict()

    if self.embeddings:
      self.embeddings_matrix = np.empty([self.vocab_max,
                                         self.embedding_size])

    id = 0
    for doc in corpus:
      for word in self.tokenizer(doc):
        word_ = word.lower()
        if (word_ in self.vocab) and (not word_ in self.w2id):
          self.w2id[word_] = id
          self.id2w[id] = word_

          # Si se aporta una matriz de embeddings,
          # aquí se crea la submatriz.
          if self.embeddings:
            if word_ in self.embeddings:
              self.embeddings_matrix[id] = self.embeddings[word_]
            else:
              self.embeddings_matrix[id] = np.random.rand(
                self.embedding_size)

          id += 1

    # Añadirmos los tokens especiales a los diccionarios.
    self.w2id.update(
      {self.UNK: id,
       self.SOS: id + 1,
       self.EOS: id + 2}
    )
    self.id2w.update(
      {id: self.UNK,
       id + 1: self.SOS,
       id + 2: self.EOS}
    )

  def get_ngram_doc(self, doc: str) -> list:
    """
    Devuelve una lista con n-gramas de un documento dado.

    Args:
        doc (str): Documento del que se quieren obtener los n-gramas.

    Returns:
        list: Lista de n-gramas.
    """
    doc_tokens = self.tokenizer(doc)
    doc_tokens = self.replace_unk(doc_tokens)
    doc_tokens = [w.lower()
                  for w in doc_tokens]
    doc_tokens = [self.SOS] * (self.N - 1) + doc_tokens + [self.EOS]

    return list(nltk.ngrams(doc_tokens, self.N))

  def replace_unk(self, doc_tokens: list[str]) -> list:
    """
    Toma un lista de tokens e intercambia los tokens out-of-vocabulary por el token especial self.UNK.

    Args:
        doc_tokens (list[str]): Lista de tokens.

    Returns:
        list: Lista de tokens procesada.
    """
    for i, token in enumerate(doc_tokens):
      if token.lower() not in self.vocab:
        doc_tokens[i] = self.UNK
    return doc_tokens

  def transform(self, corpus: list[str]) -> tuple[np.ndarray, np.ndarray]:
    """
    Devuelve una tupla de arreglos de Numpy. El primero tendrá los ids de las palabras en el contexto, mientras que la segunda el id de la palabra que se debe predecir.

    Se piensa en un modelo de CBOW. Damos el contexto y queremos predecir la palabra que sigue.

    Args:
        corpus (list[str]): Lista de documentos.

    Returns:
        tuple[np.ndarray, np.ndarray]: Arreglos de numpy con ids de los contextos y id de la palabra objetivo.
    """
    X_ngrams = list()
    y = []

    for doc in corpus:
      doc_ngram = self.get_ngram_doc(doc)
      for words_window in doc_ngram:
        words_window_ids = [self.w2id[w]
                            for w in words_window]
        X_ngrams.append(list(words_window_ids[:-1]))
        y.append(words_window_ids[-1])

    return np.array(X_ngrams), np.array(y)

Transformamos nuestros datos usando la clase `NgramData`


In [8]:
tk = TweetTokenizer()

ngram_data = NgramData(args.N, 5000, tk.tokenize, embs_dict)
ngram_data.fit(X_train)
X_ngram_train, y_ngram_train = ngram_data.transform(X_train)
X_ngram_val, y_ngram_val = ngram_data.transform(X_val)

In [97]:
print(f"Tamaño del vocabulario : {ngram_data.get_vocab_size()}")

Tamaño del vocabulario : 5000


Se crean los objetos `TensorDataset` para guardar los datos de entrenamiento y validación.


In [9]:
# Batch size
args.batch_size = 64

# Num of workers
args.num_workers = 2

# Train
train_dataset = TensorDataset(torch.tensor(X_ngram_train, dtype=torch.int64),
                              torch.tensor(y_ngram_train, dtype=torch.int64))

train_loader = DataLoader(train_dataset,
                          batch_size=args.batch_size,
                          num_workers=args.num_workers,
                          shuffle=True)

# Val
val_dataset = TensorDataset(torch.tensor(X_ngram_val, dtype=torch.int64),
                            torch.tensor(y_ngram_val, dtype=torch.int64))

val_loader = DataLoader(val_dataset,
                        batch_size=args.batch_size,
                        num_workers=args.num_workers,
                        shuffle=True)

In [99]:
batch = next(iter(train_loader))
print(f'X shape : {batch[0].shape}')
print(f'y shape : {batch[1].shape}')

X shape : torch.Size([64, 3])
y shape : torch.Size([64])


In [8]:
class NeuralLanguageModel(nn.Module):
  """
  Red neuronal de Bengio :)
  """

  def __init__(self, args, embeddings=None):
    """
    Constructor  de la clase.

    El modelo de red neuronal par lenguaje de Bengio tiene la siguiente estructura:
    Para un modelo de n-gramas, se dan las primeras n-1 palabras como contexto y se intenta predecir la n-ésima palabra.
    (1) n-1 representaciones iniciales: suelen ser one-hot. Pero aquí se toman de NgramData.
        x
    (2) n-1 representaciones aprendidas de tamaño m: se obtienen de manera individual (por palabra). 
        (x = Cx)
        En esta implementación C se inicia de manera aleatoria.
    (3) Capa oculta de tamaño h: se mezclan las n-1 representaciones del paso anterior y se aplica tanh. 
        (h = tanh(Hx + d))
        Nosotros vamos a usar ReLu en vez de tanh.
    (4) Capa de salida de tamaño m: se aplica softmax a la salida de la capa anterior.
        (y = softmax(Uh + b))
        Nosotros no vamos a aplicar softmax aquí, sino afuerita.

    Args:
        args (Any): Diccionario de variables.
    """
    super(NeuralLanguageModel, self).__init__()

    self.window_size = args.N - 1  # n-1 palabras de contexto
    self.embedding_size = args.m  # tamaño de las representaciones

    if embeddings is None:
      self.emb = nn.Embedding(args.vocab_size, args.m)
    else:
      embeddings = torch.from_numpy(embeddings).float()
      self.emb = nn.Embedding.from_pretrained(embeddings, freeze=False)
    # primera capa oculta
    self.fc1 = nn.Linear(args.m * (args.N - 1), args.d_h)
    self.drop1 = nn.Dropout(p=args.dropout)
    # producto por la matriz U.
    # softmax aplica fuera de la red
    self.fc2 = nn.Linear(args.d_h, args.vocab_size, bias=False)

  def forward(self, x):
    x = self.emb(x)
    # cambia el tamaño para que se considere como una sola capa
    x = x.view(-1, self.window_size * self.embedding_size)
    # relu(Hx + d)
    h = F.relu(self.fc1(x))  # relu(z) = max{0, z}
    h = self.drop1(h)

    # devuelve solamente (Uh + b)
    return self.fc2(h)

In [9]:
def get_preds(raw_logits: torch.Tensor) -> torch.Tensor:
  """
  Salida de la red neuronal (las neuronas de la última capa oculta).
  Uh + b
  Se les aplica la softmax
  softmax(Uh + b)
  Y luego se devuelve el índice de la neurona de mayor valor.

  Args:
      raw_logits (torch.Tensor | float): La salida de la red (Uh + b)

  Returns:
      torch.Tensor | int: Índice de la neurona con mayor valor después de softmax.
  """
  probs = F.softmax(raw_logits.detach(), dim=1)
  y_pred = torch.argmax(probs, dim=1).cpu().numpy()

  return y_pred

In [10]:
def model_eval(data: torch.Tensor,
               model,
               gpu: bool = False) -> float | int:
  """
  Evalúa el desempeño del modelo sobre un conjunto de validación.

  Args:
      data (torch.Tensor): Conjunto de validación sobre el que se va a evaluar el modelo.
      model (_type_): Modelo que se va a evaluar.
      gpu (bool, optional): ¿Usamos gpu? Sí/No. Defaults to False.

  Returns:
      float | int: Puntaje de accuracy sobre todo el conjunto de validación.
  """
  with torch.no_grad():
    preds, tgts = list(), list()

    for window_words, labels in data:
      if gpu:
        window_words = window_words.cuda()

      outputs = model(window_words)

      y_pred = get_preds(outputs)

      tgt = labels.numpy()
      tgts.append(tgt)
      preds.append(y_pred)

  # desempaquetado targets y predicciones
  # e : element
  # l : list
  # lista de todos los target
  tgts = [e for l in tgts for e in l]
  # lista de todas las predicciones
  preds = [e for l in preds for e in l]

  return accuracy_score(tgts, preds)

In [11]:
def save_checkpoint(state: dict,
                    is_best: bool,
                    checkpoint_path: str,
                    filename: str = "checkpoint.pt"):
  """
  Guarda el modelo si se ve una mejora evidente.

  Args:
      state (dict): Información que queremos guardar sobre el modelo.
        DEBE incluir model.state_dict()
      is_best (bool): ¿Esta versión del modelo es mejor?
      checkpoint_path (str): Directorio en el que se va a guardar el modelo.
      filename (str, optional): _description_. Defaults to "checkpoint.pt".
  """
  filename = os.path.join(checkpoint_path, filename)
  torch.save(state, filename)

  if is_best:
    shutil.copyfile(filename, os.path.join(checkpoint_path, "model_best.pt"))

In [104]:
# modelo
args.vocab_size = ngram_data.get_vocab_size()
args.m = 100  # dimensión de los embeddings de palabras
args.d_h = 200  # dimensión de la capa oculta
args.dropout = 0.1

# entrenamiento
args.lr = 2.3e-1
args.num_epochs = 100
args.patience = 20

# scheduler
args.lr_patience = 10
args.lr_factor = 0.5

# guardado de los modelos
args.savedir = 'model'
os.makedirs(args.savedir, exist_ok=True)

In [105]:
model = NeuralLanguageModel(args, ngram_data.embeddings_matrix)

args.use_gpu = torch.cuda.is_available()
if args.use_gpu:
  model.cuda()

# Perdida, optimización y scheduler
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),
                            lr=args.lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
  optimizer,
  "min",
  patience=args.lr_patience,
  verbose=True,
  factor=args.lr_factor
)

In [106]:
start_time = time.time()

best_metric = 0
metric_history = []
train_metric_history = []

for epoch in tqdm(range(args.num_epochs),
                  desc="Epochs"):
  epoch_start_time = time.time()
  loss_epoch = []
  training_metric = []
  model.train()

  for window_words, labels in train_loader:

    # Si hay GPU
    if args.use_gpu:
      window_words = window_words.cuda()
      labels = labels.cuda()

    # Forward
    outputs = model(window_words)
    loss = criterion(outputs,
                     labels)
    loss_epoch.append(loss.item())

    # Obtener métricas de entrenamiento
    y_pred = get_preds(outputs)
    tgt = labels.cpu().numpy()
    training_metric.append(accuracy_score(tgt,
                                          y_pred))

    # Backprop y optimizamos
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # Guardamos la métrica por época
  mean_epoch_metric = np.mean(training_metric)
  train_metric_history.append(mean_epoch_metric)

  # Validación para esta época
  model.eval()
  tuning_metric = model_eval(val_loader,
                             model,
                             gpu=args.use_gpu)
  metric_history.append(mean_epoch_metric)

  # Scheduler
  scheduler.step(tuning_metric)

  # Revisa si la métrica mejoró
  is_improvement = tuning_metric > best_metric
  if is_improvement:
    best_metric = tuning_metric
    n_no_improve = 0
  else:
    n_no_improve += 1

  # Si la métrica mejora, guarda el mejor modelo
  save_checkpoint(
    {
      "epoch": epoch + 1,
      "state_dict": model.state_dict(),
      "optimizer": optimizer.state_dict(),
      "scheduler": scheduler.state_dict(),
      "best_metric": best_metric
    },
    is_improvement,
    args.savedir
  )

  # Parada temprana por paciencie
  if n_no_improve >= args.patience:
    print("No improvement. Breaking out of loop.")
    break

  print(f"Train acc: {mean_epoch_metric}")
  print(f"Epoch [{epoch + 1}/{args.num_epochs}], Loss {np.mean(loss_epoch):.4f} - Val accuracy {tuning_metric:.4f} - Epoch time : {time.time() - epoch_start_time}")

print(f"--- {time.time() - start_time} seconds")

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Train acc: 0.15171025740167918
Epoch [1/100], Loss 5.6533 - Val accuracy 0.1989 - Epoch time : 4.217372179031372
Train acc: 0.166273525689953
Epoch [2/100], Loss 5.1789 - Val accuracy 0.1797 - Epoch time : 4.123731374740601
Train acc: 0.17133330655204274
Epoch [3/100], Loss 4.9737 - Val accuracy 0.2265 - Epoch time : 4.187483787536621
Train acc: 0.17695612220302895
Epoch [4/100], Loss 4.8127 - Val accuracy 0.2183 - Epoch time : 4.01511287689209
Train acc: 0.18040086570521832
Epoch [5/100], Loss 4.6875 - Val accuracy 0.2035 - Epoch time : 4.087488651275635
Train acc: 0.18227921755111878
Epoch [6/100], Loss 4.5652 - Val accuracy 0.1178 - Epoch time : 4.075319051742554
Train acc: 0.18446701884063793
Epoch [7/100], Loss 4.4631 - Val accuracy 0.1364 - Epoch time : 4.034951686859131
Train acc: 0.18700695225565417
Epoch [8/100], Loss 4.3649 - Val accuracy 0.1566 - Epoch time : 3.877821683883667
Train acc: 0.18931213092033905
Epoch [9/100], Loss 4.2828 - Val accuracy 0.2079 - Epoch time : 3.94

## 10 palabras mas similares

In [107]:
def print_closest_words(embeddings, ngram_data, word, n):
    '''Devuelve la lista de las n palabras mas cercanas a word'''
    word_id = torch.LongTensor([ngram_data.w2id[word]]) # obtener id de las palabras
    word_embed = embeddings(word_id) # obtener el embedding de la palabra
    dists = torch.norm(embeddings.weight - word_embed, dim=1).detach() # calcular distancias a todas las palabras
    lst = sorted(enumerate(dists.numpy()), key=lambda x: x[1]) # ordenar por distancia
    for idx, difference in lst[1:n+1]:
        print(ngram_data.id2w[idx], difference)

In [ ]:
best_model = NeuralLanguageModel(args)
state_dict = torch.load("model/model_best.pt", weights_only=False)
best_model.load_state_dict(state_dict["state_dict"])
best_model.train(False)

words = ["perro", "hijo", "casa"]
for w in words:
    print("-"*30)
    print(f"Closest words to '{w}'")
    print("-"*30)
    print_closest_words(best_model.emb, ngram_data, w, 10)

------------------------------
Closest words to 'perro'
------------------------------
gato 9.527481
perrito 12.485804
enano 17.225166
gordo 17.396631
niño 17.556175
macho 17.792421
loro 18.209782
chamaco 18.251602
burro 18.331388
caballo 18.47935
------------------------------
Closest words to 'hijo'
------------------------------
padre 16.005705
hija 16.08349
marido 16.089521
esposo 16.684036
tío 17.627182
papá 18.268936
hermano 18.30198
abuelito 18.34545
hermanito 19.020254
esposa 19.2075
------------------------------
Closest words to 'casa'
------------------------------
oficina 22.108376
depa 22.856125
cuarto 23.119127
cama 23.606852
uni 24.154474
chamba 25.643435
mochila 25.767996
heladera 26.069523
moto 26.314129
camioneta 26.537226


## Texto a partir de secuencias

In [76]:
def parse_text(text, tokenizer, ngram_data):
    '''Devuelve el texto tokenizado y los ids de las palabras'''
    all_tokens = [w.lower() if w in ngram_data.w2id else '<unk>' for w in tokenizer.tokenize(text)]
    token_ids = [ngram_data.w2id[w.lower()] for w in all_tokens]
    return all_tokens, token_ids

In [77]:
def sample_next_word(logits, temperature=1.0):
    '''
    Dados los logits y la temperatura 
    (un parametro de diversidad que indica cuan determinista sera el modelo), 
    devuelve la siguiente palabra
    '''
    logits = np.asarray(logits).astype('float64')

    preds = logits/temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probs = np.random.multinomial(1, preds)
    return np.argmax(probs)

In [83]:
def predict_next_token(model, token_ids_tensor):
    y_raw_pred = model(token_ids_tensor).squeeze(0).detach().cpu().numpy()

    y_pred = sample_next_word(y_raw_pred, 1.0)

    return y_pred


In [89]:
def generate_text(model, initial_text, tokenizer, ngram_data, max_length=100, parser=parse_text, joiner=" "):
    all_tokens, window_word_ids = parser(initial_text, tokenizer, ngram_data)
    device = next(model.parameters()).device

    for i in range(max_length):
        window_tensor = torch.tensor(window_word_ids, dtype=torch.long, device=device).unsqueeze(0)

        y_pred = predict_next_token(model, window_tensor)
        next_word = ngram_data.id2w[y_pred]
        all_tokens.append(next_word)

        if next_word == '</s>':
            break
        else:
            window_word_ids.pop(0)
            window_word_ids.append(y_pred)

    return joiner.join(all_tokens)


El modelo genera oraciones con partes coherentes, pero esto ayudara mas para calcular la probabilidad de ocurrencia de oraciones dadas.

In [ ]:
initial_tokens = '<s> <s> <s>'
print('-'*30)
print("Learned embeddings")
print('-'*30)
print(generate_text(best_model, initial_tokens, tk, ngram_data))

------------------------------
Learned embeddings
------------------------------
<s> <s> <s> si <unk> me vuelvo loca <unk> pero en estos <unk> <unk> <unk> y mi madre también yo <unk> es <unk> <unk> <unk> qué derechos de ser <unk> <unk> porque me siento si me <unk> a huevo <unk> <unk> que esta <unk> <unk> <unk> <unk> 🙄 <unk> <unk> <unk> no hay otra tipa por mi que querían <unk> la puta madre <unk> padre <unk> <unk> <unk> </s>


In [ ]:
initial_tokens = '<s> <s> <s>'
print('-'*30)
print("Learned embeddings")
print('-'*30)
print(generate_text(best_model, initial_tokens, tk, ngram_data))

------------------------------
Learned embeddings
------------------------------
<s> <s> <s> lo importante es ” ... pero mi vida en mujeres <unk> pero hasta hasta la verga o para que digan mañana <unk> pero <unk> :( </s>


In [ ]:
initial_tokens = '<s> <s> estoy'
print('-'*30)
print("Learned embeddings")
print('-'*30)
print(generate_text(best_model, initial_tokens, tk, ngram_data))

------------------------------
Learned embeddings
------------------------------
<s> <s> estoy cagada <unk> </s>


In [ ]:
initial_tokens = '<s> saludos a'
print('-'*30)
print("Learned embeddings")
print('-'*30)
print(generate_text(best_model, initial_tokens, tk, ngram_data))

------------------------------
Learned embeddings
------------------------------
<s> saludos a vergazos tiene más cualidades del <unk> que otras cansado saben <unk> <unk> <unk> <unk> 🎶 🖕🏻 la noche triste a un lado mío va sentado un mejor del <unk> a menos las <unk> </s>


## Log likelihood

In [102]:
def log_likelihood(model, test, ngram_model):
    # Generar n gram windows from input text and the respective label y
    X, y = ngram_model.transform(test)
    # discard first two n-gram windows since they may contain <s> tokens (not necessary)
    X, y = X[2:], y[2:]
    X = torch.LongTensor(X).unsqueeze(0)

    logits = model(X).detach()
    probs = F.softmax(logits, dim=1).numpy()

    return np.sum(np.log(probs[i][w]+1e-10) for i, w in enumerate(y))

Se observa como cambian los valores para oraciones menos probables de observar

In [32]:
print(f"{'Log Likelihood':<20} | Frase")
print("-" * 60)

frases = [
    "Estamos en la clase de procesamiento de lenguaje",
    "la natural Estamos clase en de de lenguaje procesamiento",
    "eres el mejor de los politicos",
    "yo desde que lo recuerdo ha estado en el",
    "siempre he creo en que estado lugar este"
]

for frase in frases:
    log_prob = log_likelihood(best_model, frase, ngram_data)
    print(f"{log_prob:<20.4f} | {frase}")


Log Likelihood       | Frase
------------------------------------------------------------
-1091.6588           | Estamos en la clase de procesamiento de lenguaje
-1271.3441           | la natural Estamos clase en de de lenguaje procesamiento
-709.9353            | eres el mejor de los politicos
-828.6460            | yo desde que lo recuerdo ha estado en el
-927.9969            | siempre he creo en que estado lugar este


/tmp/ipykernel_5636/971388848.py:11: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return np.sum(np.log(probs[i][w]) for i, w in enumerate(y))


## Permutacion de estructuras sintacticas

In [55]:
from itertools import permutations
from random import shuffle

word_list = "si no dejas de decir eso".split(" ")
perms = [' '.join(p) for p in permutations(word_list)]
#print(perms)

print('-'*30)
print("Mejores secuencias")
print('-'*30)
for p, t in sorted([(log_likelihood(best_model, text, ngram_data), text) for text in perms], reverse=True)[:5]:
    print(p, t)

print('-'*30)
print("Peores secuencias")
print('-'*30)
for p, t in sorted([(log_likelihood(best_model, text, ngram_data), text) for text in perms], reverse=True)[-5:]:
    print(p, t)

------------------------------
Mejores secuencias
------------------------------


/tmp/ipykernel_5636/971388848.py:11: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return np.sum(np.log(probs[i][w]) for i, w in enumerate(y))


-535.8627253770828 si no eso dejas decir de
-535.8627253770828 si no eso dejas de decir
-535.8627253770828 si no eso decir dejas de
-535.8627253770828 si no eso decir de dejas
-535.8627253770828 si no eso de dejas decir
------------------------------
Peores secuencias
------------------------------
-544.7678281068802 de decir dejas si eso no
-544.7678281068802 de decir dejas no si eso
-544.7678281068802 de decir dejas no eso si
-544.7678281068802 de decir dejas eso si no
-544.7678281068802 de decir dejas eso no si


## Perplejidad

In [110]:
non_pretrained_model = NeuralLanguageModel(args)

args.use_gpu = torch.cuda.is_available()
if args.use_gpu:
  non_pretrained_model.cuda()

# Perdida, optimización y scheduler
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(non_pretrained_model.parameters(),
                            lr=args.lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
  optimizer,
  "min",
  patience=args.lr_patience,
  verbose=True,
  factor=args.lr_factor
)

start_time = time.time()

best_metric = 0
metric_history = []
train_metric_history = []

for epoch in tqdm(range(args.num_epochs),
                  desc="Epochs"):
  epoch_start_time = time.time()
  loss_epoch = []
  training_metric = []
  non_pretrained_model.train()

  for window_words, labels in train_loader:

    # Si hay GPU
    if args.use_gpu:
      window_words = window_words.cuda()
      labels = labels.cuda()

    # Forward
    outputs = non_pretrained_model(window_words)
    loss = criterion(outputs,
                     labels)
    loss_epoch.append(loss.item())

    # Obtener métricas de entrenamiento
    y_pred = get_preds(outputs)
    tgt = labels.cpu().numpy()
    training_metric.append(accuracy_score(tgt,
                                          y_pred))

    # Backprop y optimizamos
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # Guardamos la métrica por época
  mean_epoch_metric = np.mean(training_metric)
  train_metric_history.append(mean_epoch_metric)

  # Validación para esta época
  non_pretrained_model.eval()
  tuning_metric = model_eval(val_loader,
                             non_pretrained_model,
                             gpu=args.use_gpu)
  metric_history.append(mean_epoch_metric)

  # Scheduler
  scheduler.step(tuning_metric)

  # Revisa si la métrica mejoró
  is_improvement = tuning_metric > best_metric
  if is_improvement:
    best_metric = tuning_metric
    n_no_improve = 0
  else:
    n_no_improve += 1

  # Si la métrica mejora, guarda el mejor modelo
  save_checkpoint(
    {
      "epoch": epoch + 1,
      "state_dict": non_pretrained_model.state_dict(),
      "optimizer": optimizer.state_dict(),
      "scheduler": scheduler.state_dict(),
      "best_metric": best_metric
    },
    is_improvement,
    args.savedir
  )

  # Parada temprana por paciencie
  if n_no_improve >= args.patience:
    print("No improvement. Breaking out of loop.")
    break

  print(f"Train acc: {mean_epoch_metric}")
  print(f"Epoch [{epoch + 1}/{args.num_epochs}], Loss {np.mean(loss_epoch):.4f} - Val accuracy {tuning_metric:.4f} - Epoch time : {time.time() - epoch_start_time}")

print(f"--- {time.time() - start_time} seconds")

best_non_pretrained_model = NeuralLanguageModel(args)
state_dict = torch.load("model/model_best.pt", weights_only=False)
best_non_pretrained_model.load_state_dict(state_dict["state_dict"])
best_non_pretrained_model.train(False)

/home/juancho/.local/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Train acc: 0.17339839513116134
Epoch [1/100], Loss 5.5124 - Val accuracy 0.2233 - Epoch time : 4.48490047454834
Train acc: 0.1832508762503515
Epoch [2/100], Loss 5.0737 - Val accuracy 0.1582 - Epoch time : 4.1563568115234375
Train acc: 0.19087852257662796
Epoch [3/100], Loss 4.8655 - Val accuracy 0.2261 - Epoch time : 4.294549465179443
Train acc: 0.19665700447917087
Epoch [4/100], Loss 4.6912 - Val accuracy 0.2313 - Epoch time : 4.221453905105591
Train acc: 0.19743470805045596
Epoch [5/100], Loss 4.5500 - Val accuracy 0.1127 - Epoch time : 3.9980862140655518
Train acc: 0.2003164167637488
Epoch [6/100], Loss 4.4173 - Val accuracy 0.2220 - Epoch time : 4.287186145782471
Train acc: 0.20291409733660062
Epoch [7/100], Loss 4.2956 - Val accuracy 0.2052 - Epoch time : 4.186666488647461
Train acc: 0.20578419384967658
Epoch [8/100], Loss 4.1773 - Val accuracy 0.2038 - Epoch time : 4.195430755615234
Train acc: 0.20922046358414012
Epoch [9/100], Loss 4.0664 - Val accuracy 0.1403 - Epoch time : 4.

NeuralLanguageModel(
  (emb): Embedding(5000, 100)
  (fc1): Linear(in_features=300, out_features=200, bias=True)
  (drop1): Dropout(p=0.1, inplace=False)
  (fc2): Linear(in_features=200, out_features=5000, bias=False)
)

In [ ]:
def perplexity(model, corpus, ngram_model):
    total_log_likelihood = 0
    total_tokens = 0

    for frase in corpus:
        X, y = ngram_model.transform(frase)
        
        X, y = X[2:], y[2:]

        if len(X) == 0:
            continue

        X_tensor = torch.LongTensor(X)
        logits = model(X_tensor).detach()
        probs = F.softmax(logits, dim=1).numpy()

        frase_log_likelihood = np.sum(np.log([probs[i][w] + 1e-10 for i, w in enumerate(y)]))
        total_log_likelihood += frase_log_likelihood

        total_tokens += len(y)

    return np.exp(-total_log_likelihood / total_tokens)

In [114]:
print(f"Perplejidad (no preentrenado): {perplexity(best_non_pretrained_model, X_val, ngram_data)}")
print(f"Perplejidad (preentrenado): {perplexity(best_model, X_val, ngram_data)}")
print("Perplejidad (probabilistico): 385.60")

Perplejidad (no preentrenado): 2208.4185075224887
Perplejidad (preentrenado): 1755.5009952801508
Perplejidad (probabilistico): 385.60


Pese a los resultados obtenidos por los NLMs entrenados aqui al encontrar palabras similares (muchas pertenecientes a algun grupo sintactico, p. ej., varias de las mas parecidas a "casa" son lugares) y al generar texto (encontrando sub-secuencias coherentes con bastante regularidad), la perplejidad calcujada es extremadamente alta, seguramente debido a un error en su calculo.

Por su parte, la accuracy alcanzada por el NLM sin embeddings preentrenados es considerablemente mas alta que la alcanzada por el NLM con embeddings preentrenados, probablemente debido a que los embeddings aleatorios generados se encuentran mas cerca de un mejor optimo que en el primer caso, de manera que obtiene mejores resultados.

# NLM a nivel caracter
Se define un tokenizer que separe los strings por caracter en lugar de por palabras. Dado que el vocabulario es mucho menor (las letras del abecedario y algunos simbolos), se adaptan los parametros para entrenar el modelo, pudiendo usarse las clases definidas anteriormente con solamente estas modificaciones (dado que el modelo a nivel caracter, FastText, es muy similar a los NLMs de Bengio).

In [54]:
def char_tokenizer(text):
    return list(text)

In [55]:
args.N = 6
vocab_max_size = 100

char_ngram_data = NgramData(args.N, vocab_max_size, char_tokenizer, embs_dict)
char_ngram_data.fit(X_train)

X_ngram_train, y_ngram_train = char_ngram_data.transform(X_train)
X_ngram_val, y_ngram_val = char_ngram_data.transform(X_val)

# Batch size
args.batch_size = 64

# Num of workers
args.num_workers = 2

# Train
train_dataset = TensorDataset(torch.tensor(X_ngram_train, dtype=torch.int64),
                              torch.tensor(y_ngram_train, dtype=torch.int64))

train_loader = DataLoader(train_dataset,
                          batch_size=args.batch_size,
                          num_workers=args.num_workers,
                          shuffle=True)

# Val
val_dataset = TensorDataset(torch.tensor(X_ngram_val, dtype=torch.int64),
                            torch.tensor(y_ngram_val, dtype=torch.int64))

val_loader = DataLoader(val_dataset,
                        batch_size=args.batch_size,
                        num_workers=args.num_workers,
                        shuffle=True)

args.vocab_size = char_ngram_data.get_vocab_size()

# Embeddings
args.m = 32
args.d_h = 64
args.dropout = 0.1

# Entrenamiento
args.lr = 0.01
args.num_epochs = 100
args.patience = 10

# Scheduler
args.lr_patience = 5
args.lr_factor = 0.5

# guardado de los modelos
args.savedir = 'char_model'
os.makedirs(args.savedir, exist_ok=True)

In [56]:
char_model = NeuralLanguageModel(args)

args.use_gpu = torch.cuda.is_available()
if args.use_gpu:
  char_model.cuda()

# Perdida, optimización y scheduler
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(char_model.parameters(),
                            lr=args.lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
  optimizer,
  "min",
  patience=args.lr_patience,
  verbose=True,
  factor=args.lr_factor
)

start_time = time.time()

best_metric = 0
metric_history = []
train_metric_history = []

for epoch in tqdm(range(args.num_epochs),
                  desc="Epochs"):
  epoch_start_time = time.time()
  loss_epoch = []
  training_metric = []
  char_model.train()

  for window_words, labels in train_loader:

    # Si hay GPU
    if args.use_gpu:
      window_words = window_words.cuda()
      labels = labels.cuda()

    # Forward
    outputs = char_model(window_words)
    loss = criterion(outputs,
                     labels)
    loss_epoch.append(loss.item())

    # Obtener métricas de entrenamiento
    y_pred = get_preds(outputs)
    tgt = labels.cpu().numpy()
    training_metric.append(accuracy_score(tgt,
                                          y_pred))

    # Backprop y optimizamos
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # Guardamos la métrica por época
  mean_epoch_metric = np.mean(training_metric)
  train_metric_history.append(mean_epoch_metric)

  # Validación para esta época
  char_model.eval()
  tuning_metric = model_eval(val_loader,
                             char_model,
                             gpu=args.use_gpu)
  metric_history.append(mean_epoch_metric)

  # Scheduler
  scheduler.step(tuning_metric)

  # Revisa si la métrica mejoró
  is_improvement = tuning_metric > best_metric
  if is_improvement:
    best_metric = tuning_metric
    n_no_improve = 0
  else:
    n_no_improve += 1

  # Si la métrica mejora, guarda el mejor modelo
  save_checkpoint(
    {
      "epoch": epoch + 1,
      "state_dict": char_model.state_dict(),
      "optimizer": optimizer.state_dict(),
      "scheduler": scheduler.state_dict(),
      "best_metric": best_metric
    },
    is_improvement,
    args.savedir
  )

  # Parada temprana por paciencie
  if n_no_improve >= args.patience:
    print("No improvement. Breaking out of loop.")
    break

  print(f"Train acc: {mean_epoch_metric}")
  print(f"Epoch [{epoch + 1}/{args.num_epochs}], Loss {np.mean(loss_epoch):.4f} - Val accuracy {tuning_metric:.4f} - Epoch time : {time.time() - epoch_start_time}")

print(f"--- {time.time() - start_time} seconds")

/home/juancho/.local/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Train acc: 0.30969145610700893
Epoch [1/100], Loss 2.4545 - Val accuracy 0.3640 - Epoch time : 32.57049298286438
Train acc: 0.36758173141830497
Epoch [2/100], Loss 2.1349 - Val accuracy 0.3922 - Epoch time : 34.05653119087219
Train acc: 0.38612480657590015
Epoch [3/100], Loss 2.0611 - Val accuracy 0.4062 - Epoch time : 32.86782670021057
Train acc: 0.39696045061324525
Epoch [4/100], Loss 2.0178 - Val accuracy 0.4136 - Epoch time : 31.996078491210938
Train acc: 0.40467761784590456
Epoch [5/100], Loss 1.9900 - Val accuracy 0.4213 - Epoch time : 33.48063540458679
Train acc: 0.4100258266608509
Epoch [6/100], Loss 1.9703 - Val accuracy 0.4253 - Epoch time : 32.648488998413086
Train acc: 0.4144382328047273
Epoch [7/100], Loss 1.9541 - Val accuracy 0.4285 - Epoch time : 33.18914604187012
Train acc: 0.417731148290079
Epoch [8/100], Loss 1.9404 - Val accuracy 0.4333 - Epoch time : 34.15164232254028
Train acc: 0.4197335420755834
Epoch [9/100], Loss 1.9335 - Val accuracy 0.4341 - Epoch time : 22.4

In [57]:
best_char_model = NeuralLanguageModel(args)
state_dict = torch.load("char_model/model_best.pt", weights_only=False)
best_char_model.load_state_dict(state_dict["state_dict"])
best_char_model.train(False)

NeuralLanguageModel(
  (emb): Embedding(100, 32)
  (fc1): Linear(in_features=160, out_features=64, bias=True)
  (drop1): Dropout(p=0.1, inplace=False)
  (fc2): Linear(in_features=64, out_features=100, bias=False)
)

## Generacion de texto

In [85]:
def parse_text_char(text, tokenizer, ngram):
    '''Devuelve el texto tokenizado y los ids de caracteres'''
    all_tokens = [w.lower() if w.lower() in ngram.w2id else '<unk>' for w in tokenizer(text)]
    token_ids = [ngram.w2id[w] for w in all_tokens]
    window_size = ngram.N - 1
    token_ids = token_ids[-window_size:]
    return all_tokens, token_ids

In [90]:
print(generate_text(char_model, "hola h", char_tokenizer, char_ngram_data, max_length=300, parser=parse_text_char, joiner=""))
print(generate_text(char_model, "ellas ", char_tokenizer, char_ngram_data, max_length=300, parser=parse_text_char, joiner=""))
print(generate_text(char_model, "vayans", char_tokenizer, char_ngram_data, max_length=300, parser=parse_text_char, joiner=""))

hola hey <unk>mona mandemos mi en ula sechetelos persos los mamán ywvada derente espero a mejo que mamisiate arigre<unk></s>
ellas putas gú de madran pinan mander madida feas y la verga  chas y lo quino ma el cocoragadar la vay paña verga más me que la vizez <unk><unk> asique esea de amos que bus as<unk> hoberten gata <unk>va fr de no le fan😂s<unk></s>
vayansap pricieliriaca <unk>ménima ceapelada uecon<unk></s>


## Log likelihood

In [103]:
print(f"{'Log Likelihood':<20} | Frase")
print("-" * 60)

frases = [
    "Estamos en la clase de procesamiento de lenguaje",
    "la natural Estamos clase en de de lenguaje procesamiento",
    "eres el mejor de los politicos",
    "yo desde que lo recuerdo ha estado en el",
    "siempre he creo en que estado lugar este"
]

for frase in frases:
    log_prob = log_likelihood(best_char_model, frase, char_ngram_data)
    print(f"{log_prob:<20.4f} | {frase}")


Log Likelihood       | Frase
------------------------------------------------------------
-797.1431            | Estamos en la clase de procesamiento de lenguaje
-937.4966            | la natural Estamos clase en de de lenguaje procesamiento
-494.0180            | eres el mejor de los politicos
-683.9063            | yo desde que lo recuerdo ha estado en el
-680.1800            | siempre he creo en que estado lugar este


/tmp/ipykernel_7188/3418010109.py:11: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return np.sum(np.log(probs[i][w]+1e-10) for i, w in enumerate(y))


## Estructura morfologica

In [105]:
from itertools import permutations
from random import shuffle

char_list = char_tokenizer("si dejas")
perms = [''.join(p) for p in permutations(char_list)]
#print(perms)

print('-'*30)
print("Mejores secuencias")
print('-'*30)
for p, t in sorted([(log_likelihood(best_char_model, text, char_ngram_data), text) for text in perms], reverse=True)[:5]:
    print(p, t)

print('-'*30)
print("Peores secuencias")
print('-'*30)
for p, t in sorted([(log_likelihood(best_char_model, text, char_ngram_data), text) for text in perms], reverse=True)[-5:]:
    print(p, t)

------------------------------
Mejores secuencias
------------------------------


/tmp/ipykernel_7188/3418010109.py:11: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return np.sum(np.log(probs[i][w]+1e-10) for i, w in enumerate(y))


-111.97369872578572 di sajes
-111.97369872578572 di sajes
-111.97369872578572 di asjes
-111.97369872578572 di asjes
-111.97369872578572 d isajes
------------------------------
Peores secuencias
------------------------------
-117.58164512966646 sjsd iea
-117.58164512966646 sdsj iea
-117.58164512966646 sdsj iea
-117.58164512966646 sds jiea
-117.58164512966646 sds jiea


## Perplejidad

In [106]:
print(f"Perplejidad: {perplexity(best_char_model, X_val, char_ngram_data)}")

Perplejidad: 4374.900780770517


Como en la NLM a nivel palabra, en este caso en la generacion de texto se observan secuencias de caracteres que suenan familiares en el espanol, si bien la mayoria no son palabras coherentes.

En el caso del calculo del log likelihood, se observan valores bastante mas cercanos a cero que en el caso anterior, probablemente debido a que hay una menor cantidad de secuencias posibles (cada palabra puede combinarse con otros miles de palabras, pero cada caracter solo puede combinarse con algunos cientos de ellos), y por lo tanto, una mayor probabilidad asociada a su ocurrencia.

Lo mismo sucede en las estructuras morfologicas, donde las permutaciones clasificadas como mejores poseen secuencias familiares, mientras que las peores son secuencias muy extranas o no vistas en el espanol (por ejemplo, varias consonantes o vocales consecutivas).

Finalmente, la perplejidad del modelo es la mas alta que la del resto de los modelos entrenados, probablemente debido a la informalidad del lenguaje presente en el corpus (faltas de ortografia, palabras mal escritas, variabilidad en los emojis usados, etc.), lo cual puede introducir una mayor cantidad de ruido que el presente a nivel palabra.